In [ ]:
import sys; sys.path.extend(['..', '../validations/'])
import MeshFEM
import mesh, elastic_sheet
from tri_mesh_viewer import TriMeshViewer, PointCloudViewer
import py_newton_optimizer, fd_validation, benchmark
from matplotlib import pyplot as plt
import meshing, time
import numpy as np
from io_redirection import suppress_stdout as so
import sim_utils, sheet_convergence

In [ ]:
def bend(X, angle = np.pi / 4):
    x, y, z = X[:, 0], X[:, 1], X[:, 2]
    L = x.max() - x.min()
    xm = 0.5 * (x.max() + x.min())
    height = 0.5 * L * np.sin(angle)
    z = height * 2 * (L - np.abs(xm - x)) / L
    # (s * x)^2 + (2 * H / L)^2 = 1
    x *= np.sqrt(1 - (2 * height / L)**2)
    return np.column_stack((x, y, z))

In [ ]:
es = sheet_convergence.getSheet(0.05, maxArea=1e-3, L=6, useNeoHookean=True)
es.setDeformedPositions(bend(es.getRestPositions(), 3.5 * np.pi / 8))
es.initializeMidedgeNormals()
es.energy(es.EnergyType.Membrane), es.energy(es.EnergyType.Bending)

In [ ]:
leftEdgeVars = sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MIN_X, displacementsOnly=True)
rightEdgeVars= sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MAX_X, displacementsOnly=True)
clampThetas  = sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MIN_Z, displacementComponents=[], restPos=False, tol=2.0)
botVars      = sim_utils.getBBoxVars(es, sim_utils.BBoxFace.MIN_Z, displacementsOnly=True, tol=1.0, restPos=False)

In [ ]:
v_plate = TriMeshViewer(es, wireframe=True, width=800, height=600)
v_plate.show()

In [ ]:
strainEigs = np.array([np.linalg.eigh(0.5 * (C - np.identity(2)))[0] for C in es.getC()])
v_plate.update(scalarField=strainEigs[:, 0])

In [ ]:
plate_normalVis = None
def updateNormalView():
    return
    global plate_normalVis
    v_plate.subViews = []
    plate_normalVis = PointCloudViewer(es.edgeMidpoints(), vectorField=es.midedgeNormals(), superView=v_plate)
    plate_normalVis.arrowSize = 20
updateNormalView()

In [ ]:
def iter_cb(prob, it):
    v_plate.update()
    #updateNormalView()
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 50
opts.gradTol = 1e-9
es.computeEquilibrium([], botVars + clampThetas, opts=opts, cb=iter_cb)
#v_plate.update()

In [ ]:
es.energy(), es.energy(etype=es.EnergyType.Membrane), es.energy(etype=es.EnergyType.Bending)